In [1]:
#################################
# filenames
#################################

probs_file = 'newprobs4.json'
filename3s = 'newprobs4genre.json'
filename3se = 'newprobs4e.txt'
histo_file = 'histo4.txt'
url_dict_file = 'url.json'


##############################
# setup
##############################

import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random

tstart = time.time()
print(tstart)

api_url = 'https://api.hooktheory.com/v1/'
login = {"username": "jl8429",
         "password": ""}

r = requests.post(api_url+'users/auth', data=login)
#print('r', r)
#print(r.json())
'''
{'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
'''

activkey = '0e11e1d65a021d0a883f309f813d74ad'
header = {"Authorization": "Bearer "+activkey}


#################################
# RELOAD url_dict
#################################

with open(url_dict_file, 'r') as f:
    url_dict = json.load(f)

big_dict = {}


#################################
# continue
#################################

#with open('chordProbs3new.json', "r") as f:
with open(probs_file, "r") as f:
    data3 = json.load(f)
f2 = open(filename3se, 'w')

j = 1
totals = str(len(data3))
print(totals)

histo = [0]*5000

driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")

for d in data3:
    chord1, chord2, chord3, chord4 = d['child_path'].split(",")
    mykey = str(chord1)+','+str(chord2)+','+str(chord3)+','+str(chord4)
    print(str(j)+'/'+totals, mykey)
    song_list = []
    
    # don't overwrite what we've done already
    if not big_dict.get(mykey, False):
        try:
            myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
            
            # slow down...
            time.sleep(1.1)
            if str(myData)=="<Response [429]>":
                print("too many requests")
                time.sleep(30)
            
            page = 1
            keepGoing = True
            #while myData.content and myData.content != b'No songs match this chord progressionnull':
            while keepGoing:
                # can't parse if it's this byte string
                if myData.content != b'No songs match this chord progressionnull':
                    #print('page', page)
                    # parse each song
                    try:
                        my_json = myData.json()
                        songs = len(my_json)
                        #print('songs len', songs)
                        if songs:
                            # for each song, pull the genre, add it to the url_dict and big_dict, then write it to the file
                            for i in range(songs):
                                url = my_json[i]['url'].split("#")[0]
                                #print(url)
                                # check if already in url_dict to save us from having to load the page slowly and parse
                                if url in url_dict:
                                    #print(url, url_dict[url])
                                    my_json[i]['genre'] = url_dict[url]
                                # if not in dict, we need to parse page and add to dict
                                else:
                                    driver.get(url)
                                    html = driver.page_source
                                    soup = BeautifulSoup(html, 'html.parser')
                                    try:
                                        #genre = soup.find_all(['multiselect'])[0].find('a').contents[0]
                                        # some songs don't have a genre and some have multiple genres...
                                        genres = soup.find_all(['multiselect'])[0].find_all('a')
                                        gk = 0
                                        glist = []
                                        for g in genres:
                                            # .find_all() will return a lot of garbage, but the genres will have a length of 1
                                            if len(g) > 1:
                                                # if even the first is garbage, it has no genre
                                                if gk == 0:
                                                    my_json[i]['genre'] = ["Unknown"]
                                                    url_dict[url] = ["Unknown"]
                                                break
                                            else:
                                                glist.append(g.contents[0])
                                            gk += 1
                                        my_json[i]['genre'] = glist
                                        url_dict[url] = glist
                                    except:
                                        print('3. Unable to extract genre for', url)
                                        my_json[i]['genre'] = ["Unknown"]
                                        url_dict[url] = ["Unknown"]

                                    # this is not where this block belongs
                                    # nesting it under "else" above made failures generate "[]" and 
                                    #    only appended new songs to song_list, ignoring those already in url_dict
#                                     #driver.close()
#                                     # clean up and append song to list
#                                     my_json[i].pop('section', None)
#                                     my_json[i]['url'] = url
#                                     #f.write('\n'+str(my_json[i])+',')
#                                     song_list.append(my_json[i])

                                # this is where the above block really belongs 
                                #     so song_list gets every song, even if the url is in url_dict
                                #driver.close()
                                # clean up and append song to list
                                my_json[i].pop('section', None)
                                my_json[i]['url'] = url
                                #f.write('\n'+str(my_json[i])+',')
                                #print(my_json[i])
                                song_list.append(my_json[i])

                            if songs == 20:
                                #keepGoing = True
                                page += 1
                                myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                # slow down...
                                time.sleep(1.1)
                                if str(myData)=="<Response [429]>":
                                    print("too many requests")
                                    time.sleep(30)
                            else:
                                # update histogram of how many songs per chord progression
                                index = page*20 - 20 + songs
                                print('histo1', index)
                                histo[index] += 1
                                keepGoing = False
                        else:
                            # update histogram of how many songs per chord progression
                            index = page*20 - 20
                            print("histo2", index)
                            histo[index] += 1
                            keepGoing = False

                    except:
                        # for some reason, page=20 doesn't work on 1-5-6-4... there are other data corruptions too
                        print('2. Unable to extract for', chord1, chord2, chord3, chord4, "page", page)
                        f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
                        f2.close()
                        f2 = open(filename3se, 'a')

                        page += 1
                        myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                        # slow down...
                        time.sleep(1.1)
                        if str(myData)=="<Response [429]>":
                            print("too many requests")
                            time.sleep(30)
                else:
                    print("No songs match this chord progression", mykey)
                    keepGoing = False

        except:
            print('1. Unable to extract for', chord1, chord2, chord3, chord4)
            f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
            f2.close()
            f2 = open(filename3se, 'a')

            # slow down
            time.sleep(1.1)
        
        big_dict[mykey] = song_list
    #f.close()
    #f = open(filename3s, 'a')
    #time.sleep(1.1)
    j += 1
    # restart the driver every 100 chord progressions
    if j%100==0:
        driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")
    
f2.close()


with open(filename3s, "w") as f:
    json.dump(big_dict, f)

tend = time.time()
print(tend)
#f.close()
print(tend-tstart)

print(histo)
with open(histo_file, "w") as f:  
    f.write(str(histo))

with open(url_dict_file, "w") as f:
    json.dump(url_dict, f)

1615050996.3771405
500
1/500 37,57/2,b7,47
histo1 1
2/500 164,542/4,46,b6
histo1 1
3/500 1,L27,57/3,L3
histo1 1
4/500 4,1,4,242
histo1 1
5/500 4,56,1,4
histo1 1
6/500 265,6,5,47
histo1 1
7/500 1,4,3,67
histo1 1
8/500 6,56,3,1
histo1 1
9/500 4,164,46,565
histo1 1
10/500 67,47,67,L57
histo1 1
11/500 2,5,M565,1
histo1 1
12/500 27,37,6,1
histo1 1
13/500 7,4,1,4
histo1 1
14/500 6,b7,6,b7
histo1 8
15/500 6,7,37,6
histo1 1
16/500 6,66,56,564
histo1 1
17/500 4,16,37,26
histo1 1
18/500 664,66,6,4
histo1 1
19/500 4,2,6,56
histo1 7
20/500 57/6,67,57/3,37
histo1 1
21/500 2,6,5,3
histo1 7
22/500 1,5/6,4,57/5
histo1 2
23/500 6,7/6,6,7/3
histo1 1
24/500 1,4,b3,1
histo1 3
25/500 5,4,27,67
histo1 1
26/500 47,1,5,4
histo1 1
27/500 664,4,564,26
histo1 1
28/500 1,56,1,b57
histo1 1
29/500 5,564,6,664
histo1 1
30/500 2,264,57/5,5
histo1 1
31/500 7,2,3,6
histo1 1
32/500 6,3,5,27
histo1 1
33/500 27,1,b1,1
histo1 1
34/500 2,264,6,664
histo1 2
35/500 5,1,4,46
histo1 2
36/500 1,2,b16,16
histo1 1
37/500 4,16,27,4

histo1 1
293/500 5,4,1,3
histo1 8
294/500 6,L7,4,5
histo1 1
295/500 37,6,164,L7
histo1 2
296/500 1,56,1,4
histo1 7
297/500 4,3,2,4
histo1 14
298/500 56,4,5/2,4
histo1 2
299/500 16,7,1,M142
histo1 1
300/500 665,565/2,27,67
histo1 2
301/500 16,4,67,4
histo1 2
302/500 5,37,47,27
histo1 3
303/500 6,46,L46,46
histo1 5
304/500 264,6,664,365
histo1 1
305/500 5,37,67,5
histo1 2
306/500 26,5,56/6,6
histo1 1
307/500 67,4,1,67
histo1 6
308/500 67,47,2,3
histo1 1
309/500 5/2,4,b7,1
histo1 1
310/500 67,5,1,67
histo1 2
311/500 b7,264,57,M17
histo1 1
312/500 5,47,2,1
histo1 1
313/500 5,L26,4,5
histo1 3
314/500 164,57,164,1
histo1 1
315/500 46,5,4,46
histo1 1
316/500 5/2,47,37,b37
histo1 1
317/500 26,36,57,37
histo1 1
318/500 6,5/6,1,56
histo1 1
319/500 5,3,67,4
histo1 3
320/500 1,67,6,57/5
histo1 1
321/500 1,27,242,77
histo1 1
322/500 17,27,17,27
histo1 3
323/500 164,5,564,5
histo1 1
324/500 642,6,1,2
histo1 1
325/500 3,2,664,4
histo1 1
326/500 26,5,6,5
histo1 2
327/500 56,6,57/6,47
histo1 1
328/500 

In [ ]:
##########################
# list errors here
##########################
57/500 16,564,1,56
2. Unable to extract for 16 564 1 56 page 1
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

In [3]:
##########################
# test errors here
##########################
mykey = '16,564,1,56'
page = 1
myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
#myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
print(myData.content)
#print(len(myData.json()))
#print(myData.json())

b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'


In [2]:
##########################
# fix histo here
##########################
# histo[20] -= 1
# print(histo)
# with open(histo_file, "w") as f:  
#     f.write(str(histo))

[0, 355, 74, 18, 12, 10, 5, 7, 4, 2, 3, 0, 1, 1, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [12]:
# #################################
# # create url.json to read url_dict from in future parses
# #################################

# ### don't forget to read from newprobs0genre and newprobs1genre too!

# # filenames
# filename0 = 'newprobs0genre.json'
# filename1 = 'newprobs1genre.json'
# filename2 = 'newprobs2genre.json'
# url_dict_file = 'url.json'

# with open(filename0, 'r') as fg:
#     big_dict0 = json.load(fg)
# with open(filename1, 'r') as fg:
#     big_dict1 = json.load(fg)
# with open(filename2, 'r') as fg:
#     big_dict2 = json.load(fg)

# url_dict = {}

# for bd in big_dict0:
#     if big_dict0[bd]:
#         for song in big_dict0[bd]:
#             url_dict[song['url']] = song['genre']
# for bd in big_dict1:
#     if big_dict1[bd]:
#         for song in big_dict1[bd]:
#             url_dict[song['url']] = song['genre']
# for bd in big_dict2:
#     if big_dict2[bd]:
#         for song in big_dict2[bd]:
#             url_dict[song['url']] = song['genre']

# def display_dict(d):
#     print(len(url_dict.keys()))
#     i = 0
#     for gd in url_dict:
#         print(gd, url_dict[gd])
#         if i == 10:
#             break
#         i += 1

# display_dict(url_dict)

# with open(url_dict_file, 'w') as f:
#     json.dump(url_dict, f)

# with open(url_dict_file, 'r') as f:
#     url_dict = json.load(f)

# display_dict(url_dict)

2147
http://www.hooktheory.com/theorytab/view/10cc/the-wall-street-shuffle []
http://www.hooktheory.com/theorytab/view/eric-whitacre/sleep []
http://www.hooktheory.com/theorytab/view/jason-lee/the-japanese-play-song []
http://www.hooktheory.com/theorytab/view/alabama-shakes/gimme-all-your-love ['Rock']
http://www.hooktheory.com/theorytab/view/antonin-dvorak/humoresque-op-101-no-7 ['Classical']
http://www.hooktheory.com/theorytab/view/frog-fractions-soundtrack/rimushotto-bungie-jump []
http://www.hooktheory.com/theorytab/view/the-shins/young-pilgrims []
http://www.hooktheory.com/theorytab/view/tame-impala/the-moment []
http://www.hooktheory.com/theorytab/view/bill-withers/lean-on-me ['Rock', 'Soul']
http://www.hooktheory.com/theorytab/view/disney/can-you-feel-the-love-tonight ["Children's", 'Disney']
http://www.hooktheory.com/theorytab/view/paul-simon/still-crazy-after-all-these-years ['Pop', 'Rock']
2147
http://www.hooktheory.com/theorytab/view/10cc/the-wall-street-shuffle []
http://ww